In [42]:
## Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tqdm import tqdm

In [43]:
# For reproducibility
np.random.seed(42)

In [44]:
## Function to calculate MACD
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    short_ema = data.ewm(span=short_window, adjust=False).mean()
    long_ema = data.ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    histogram = macd - signal
    return macd, signal, histogram


## Hill Climbing Algorithm
def hill_climbing(macd, signal, initial_delta1=0, initial_delta2=0, max_iter=100):
    delta1, delta2 = initial_delta1, initial_delta2
    best_profit = -np.inf
    
    for _ in range(max_iter):
        candidate_deltas = [(delta1 + np.random.randn(), delta2 + np.random.randn()) for _ in range(10)]
        profits = []
        
        for d1, d2 in candidate_deltas:
            actions = generate_actions(macd, signal, d1, d2)
            profit = calculate_profit(actions)
            profits.append((profit, d1, d2))
        
        best_candidate = max(profits, key=lambda x: x[0])
        if best_candidate[0] > best_profit:
            best_profit, delta1, delta2 = best_candidate
        else:
            break
    
    return delta1, delta2

## Function to generate actions
def generate_actions(macd, signal, delta1, delta2):
    actions = []
    for i in range(1, len(macd)):
        if macd[i] - signal[i] > delta1:
            actions.append(1)  # Buy
        elif signal[i] - macd[i] > delta2:
            actions.append(-1)  # Sell
        else:
            actions.append(0)  # Hold
    return actions

## Calculate Profit
def calculate_profit(actions):
    profit = 0
    position = 0  # 1 for holding a stock, 0 for not holding
    for action in actions:
        if action == 1 and position == 0:
            position = 1  # Buy
            buy_price = current_price
        elif action == -1 and position == 1:
            position = 0  # Sell
            profit += current_price - buy_price
    return profit

In [45]:
## GRU Model
def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(GRU(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))  # Output layer for Buy, Sell, Hold
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [51]:
## Prepare Data
def prepare_data(data, window_size=5):
    macd, signal, _ = calculate_macd(data['Close'])
    data['MACD'] = macd
    data['Signal'] = signal
    data = data.dropna()

    features = []
    labels = []
    for i in range(window_size, len(data)-10):
        features.append(data[['MACD', 'Signal', 'Close']].iloc[i-window_size:i].values)
        labels.append(generate_actions(macd, signal, 0, 0)[i])
    
    features = np.array(features)
    labels = np.array(labels)
    
    return features, labels


In [49]:
# Load your stock data here
data = pd.read_csv('D:\Stock Predicton\Datasets\MSFT.csv')
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2011-03-16,25.219999,25.280001,24.680000,24.790001,19.780724,100725400
1,2011-03-17,25.059999,25.219999,24.750000,24.780001,19.772743,62497000
2,2011-03-18,25.059999,25.180000,24.799999,24.799999,19.788700,85486700
3,2011-03-21,25.180000,25.580000,25.150000,25.330000,20.211607,46878100
4,2011-03-22,25.299999,25.459999,25.230000,25.299999,20.187656,30895600
...,...,...,...,...,...,...,...
2511,2021-03-09,232.880005,235.380005,231.669998,233.779999,233.779999,33034000
2512,2021-03-10,237.000000,237.000000,232.039993,232.419998,232.419998,29733000
2513,2021-03-11,234.960007,239.169998,234.309998,237.130005,237.130005,29896000
2514,2021-03-12,234.009995,235.820007,233.229996,235.750000,235.750000,22647900


In [52]:
# Prepare data
features, labels = prepare_data(data)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


In [53]:
# Build and train model
model = build_gru_model((X_train.shape[1], X_train.shape[2]))
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

c:\Users\g6kan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "c:\Users\g6kan\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main

  File "c:\Users\g6kan\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code

  File "C:\Users\g6kan\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\g6kan\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1043, in launch_instance

  File "C:\Users\g6kan\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 725, in start

  File "C:\Users\g6kan\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 215, in start

  File "c:\Users\g6kan\AppData\Local\Programs\Python\Python310\lib\asyncio\base_event

In [ ]:
# Evaluate model
model.evaluate(X_test, y_test)

In [ ]:

# Visualization
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
# Predict and Simulate Trading
predictions = model.predict(X_test)
predicted_actions = np.argmax(predictions, axis=1)

# Simulate trading based on predicted actions
profits = calculate_profit(predicted_actions)
print(f'Total Profit: {profits}')